In [3]:
import os
import cv2
import numpy as np
import torch
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from pathlib import Path
from tqdm import tqdm

# =========================
# PATHS
# =========================
DATA_ROOT = Path("Dataset/ShanghaiTech/part_A/train_data")
IMG_DIR = DATA_ROOT / "images"
GT_DIR  = DATA_ROOT / "ground-truth"

SAVE_DIR = Path("preprocessed/TrainA")
IMG_SAVE = SAVE_DIR / "images"
GT_SAVE  = SAVE_DIR / "gt"

IMG_SAVE.mkdir(parents=True, exist_ok=True)
GT_SAVE.mkdir(parents=True, exist_ok=True)

# =========================
# CONSTANTS
# =========================
TARGET_SIZE = (512, 512)
GT_SIZE = (64, 64)

def make_density_map(points, shape):
    h, w = shape
    density = np.zeros((h, w), np.float32)
    for x, y in points:
        if 0 <= int(x) < w and 0 <= int(y) < h:
            density[int(y), int(x)] = 1.0
    density = gaussian_filter(density, sigma=4)
    return density

# =========================
# PREPROCESS LOOP
# =========================
for img_path in tqdm(sorted(IMG_DIR.glob("*.jpg"))):
    name = img_path.stem

    # Load image
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, TARGET_SIZE).astype(np.float32) / 255.0
    img_tensor = torch.tensor(img).permute(2,0,1)

    # Load GT
    mat = loadmat(str(GT_DIR / f"GT_{name}.mat"))
    points = mat["image_info"][0][0][0][0][0]

    # Density map
    den = make_density_map(points, TARGET_SIZE)

    # Downsample for CSRNet
    den_small = cv2.resize(den, GT_SIZE) * 64
    den_tensor = torch.tensor(den_small).unsqueeze(0)

    # SAVE (MATCH DATASET LOADER)
    torch.save(img_tensor, IMG_SAVE / f"{name}.pt")
    torch.save(den_tensor, GT_SAVE / f"{name}.pt")

print("✔ TrainA preprocessing complete.")


100%|██████████| 300/300 [00:10<00:00, 27.58it/s]

✔ TrainA preprocessing complete.
